## CL Backtester Architechture

This is not formal research, but rather a notebook that aims to:
* Serve as practice
* Allow me to try architecting a backtesting workflow for futures contracts (in contrast to the equities backtesting done in class) 

In [1]:
import pandas as pd
from pathlib import Path
from datetime import timezone, timedelta
import hvplot.pandas
import numpy as np

In [2]:
pd.set_option('display.max_rows', 500)

### Bring in CL Data
Hourly data for the CL1! continuous contract is was downloaded from TradingView, and saved to csv.

In [3]:
# Read the CSV file
cl_60_df = pd.read_csv(Path('../../Data/NYMEX_CL1!, 60_af366.csv'))

# Convert 'time' column from timestamp (seconds since epoch) to actual time 
cl_60_df['time'] = pd.to_datetime(
    cl_60_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)

# Convert timezone from UTC to Eastern Time
cl_60_df['time'] = cl_60_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index
cl_60_df.set_index('time', inplace=True)

# Review df
#cl_60_df.head()


In [4]:
cl_60_df = cl_60_df.rename(columns={
                'Upper Band #1': 'vwap_upper1',
                'Upper Band #2': 'vwap_upper2',
                'Upper Band #3': 'vwap_upper3',
                'Lower Band #1': 'vwap_lower1',
                'Lower Band #2': 'vwap_lower2',
                'Lower Band #3': 'vwap_lower3',
                'EMA': '200EMA',
                'Smoothing Line': '200EMA Smoothed?',
                'Volume MA': '200VolMA'
})

In [5]:
cl_60_df

,open,high,low,close,VWAP,vwap_upper1,vwap_lower1,vwap_upper2,vwap_lower2,vwap_upper3,vwap_lower3,200EMA,200EMA Smoothed?,Volume,200VolMA
time,,,,,,,,,,,,,,,
2021-01-03 18:00:00-05:00,31.40,31.52,31.11,31.43,31.353333,31.353333,31.353333,31.353333,31.353333,31.353333,31.353333,NaN,NaN,5333,NaN
2021-01-03 19:00:00-05:00,31.43,31.47,31.37,31.39,31.369904,31.395680,31.344128,31.421457,31.318351,31.447233,31.292575,NaN,NaN,2204,NaN
2021-01-03 20:00:00-05:00,31.39,31.78,31.39,31.76,31.507591,31.645504,31.369679,31.783416,31.231767,31.921328,31.093854,NaN,NaN,7645,NaN
2021-01-03 21:00:00-05:00,31.77,32.33,31.73,32.17,31.713226,32.007986,31.418467,32.302745,31.123708,32.597505,30.828948,NaN,NaN,8590,NaN
2021-01-03 22:00:00-05:00,32.18,32.24,31.95,31.98,31.784615,32.081675,31.487555,32.378735,31.190495,32.675795,30.893435,NaN,NaN,6238,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-12 05:00:00-04:00,75.13,75.25,74.89,75.12,74.970198,75.075433,74.864963,75.180668,74.759728,75.285903,74.654493,72.299943,72.243772,8938,8938.30
2023-07-12 06:00:00-04:00,75.12,75.14,74.81,74.93,74.968721,75.066106,74.871337,75.163490,74.773953,75.260874,74.676568,72.326112,72.271057,7123,8024.95
2023-07-12 07:00:00-04:00,74.93,75.19,74.82,75.13,74.976876,75.072064,74.881689,75.167251,74.786502,75.262438,74.691314,72.354012,72.298888,5748,6544.40


### Signal Definition

In [6]:
# Setup signals_df from primary cl df
signals_df = cl_60_df[['close','VWAP','200EMA']].dropna()

#Add Placeholder signal column
signals_df['Signal'] = 0.0

#review df
signals_df

,close,VWAP,200EMA,Signal
time,,,,
2021-01-14 09:00:00-05:00,35.87,35.756447,34.091800,0.0
2021-01-14 10:00:00-05:00,35.61,35.754863,34.106906,0.0
2021-01-14 11:00:00-05:00,35.92,35.762846,34.124947,0.0
2021-01-14 12:00:00-05:00,36.27,35.794224,34.146291,0.0
2021-01-14 13:00:00-05:00,36.29,35.835392,34.167621,0.0
...,...,...,...,...
2023-07-12 05:00:00-04:00,75.12,74.970198,72.299943,0.0
2023-07-12 06:00:00-04:00,74.93,74.968721,72.326112,0.0
2023-07-12 07:00:00-04:00,75.13,74.976876,72.354012,0.0


### Trading Algo (Signal Generation)
For this example, we generate a long signal (+1) when the VWAP is greater than the 200EMA, and a short signal (-1) when the opposite is true.
As stated above, the actual strategy here is sort of immaterial. 

In [7]:
# Generate signals
#IF VWAP > 200EMA, bullish.
#IF VWAP < 200EMA, bearish.
signals_df['Signal'] = np.where(
    signals_df['200EMA'] < signals_df['VWAP'], 1.0, -1.0
)

#Determine entries
signals_df['Entry/Exit'] = signals_df['Signal'].diff()


signals_df.head(5)


### We start with a position... how can this be avoided?
#### Maybe it's fine, since it is technically correct that we would be in that position at that time



,close,VWAP,200EMA,Signal,Entry/Exit
time,,,,,
2021-01-14 09:00:00-05:00,35.87,35.756447,34.091800,1.0,NaN
2021-01-14 10:00:00-05:00,35.61,35.754863,34.106906,1.0,0.0
2021-01-14 11:00:00-05:00,35.92,35.762846,34.124947,1.0,0.0
2021-01-14 12:00:00-05:00,36.27,35.794224,34.146291,1.0,0.0
2021-01-14 13:00:00-05:00,36.29,35.835392,34.167621,1.0,0.0


In [8]:
signals_df[['Entry/Exit']].value_counts()

Entry/Exit
 0.0          14528
-2.0            103
 2.0            103
dtype: int64

In [9]:
# Visualize exit position relative to close price
short = signals_df[signals_df['Entry/Exit'] == -2.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
long = signals_df[signals_df['Entry/Exit'] == 2.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price
cl_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='CL Close $',
    width=1000,
    height=400
)

# Visualize moving averages
ema200 = signals_df[['200EMA']].hvplot(
    line_color='darkblue',
    ylabel='CL Close $',
    width=1000,
    height=400
)

# Visualize VWAP
vwap = signals_df[['VWAP']].hvplot(
    line_color='lightgreen',
    ylabel='CL Close $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = cl_close * ema200 * vwap * long * short

# Show the plot with a title
entry_exit_plot.opts(
    title="CL1 - 200EMA, VWAP"
)

:Overlay
   .Curve.I          :Curve   [time]   (close)
   .Curve.II         :Curve   [time]   (200EMA)
   .Curve.III        :Curve   [time]   (VWAP)
   .Scatter.Close.I  :Scatter   [time]   (close)
   .Scatter.Close.II :Scatter   [time]   (close)

### Performance Calculation & Evaluation

In [21]:
#Initialize variables
initial_capital = 50000
contract_name = 'CL1!'
dollars_per_tick = 10
tick_size = 0.01
number_of_contracts = 10
adjuster = dollars_per_tick * number_of_contracts // tick_size

In [11]:
# Establish Position column
signals_df['Position'] = number_of_contracts * signals_df['Signal']

# Establish Entry/Exit Position column (order size?)
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()


signals_df.head(5)

,close,VWAP,200EMA,Signal,Entry/Exit,Position,Entry/Exit Position
time,,,,,,,
2021-01-14 09:00:00-05:00,35.87,35.756447,34.091800,1.0,NaN,10.0,NaN
2021-01-14 10:00:00-05:00,35.61,35.754863,34.106906,1.0,0.0,10.0,0.0
2021-01-14 11:00:00-05:00,35.92,35.762846,34.124947,1.0,0.0,10.0,0.0
2021-01-14 12:00:00-05:00,36.27,35.794224,34.146291,1.0,0.0,10.0,0.0
2021-01-14 13:00:00-05:00,36.29,35.835392,34.167621,1.0,0.0,10.0,0.0


In [12]:
# Initialize columns to populate with for loop
signals_df['trade_price'] = 0.0
signals_df['trade_delta'] = 0.0
signals_df['trade_value'] = 0.0
signals_df['unrealized_acct_val'] = 0.0
signals_df['realized_pnl'] = 0.0
signals_df['cum_realized_acct_val'] = 0.0

# Initialize list to store execution prices
trade_price = []
realized = []

for index, row in signals_df.iterrows():
    
    # If a trade is indicated in 'Entry/Exit'
    if row['Entry/Exit'] != 0:
    
        # Set trade price as period close
        row['trade_price'] = row['close']
        
        # Append trade price to list
        trade_price.append(row['trade_price'])
    
    else:
        
        # leave at 0 (no trade)
        row['trade_price'] = 0.0
   
    # If we have a long position
    if row['Position'] > 0:
        
        # Find the unrealized LONG pnl via (Period close - most recent trade price added to trade_price list) 
        signals_df.at[index, 'trade_delta'] = (row['close'] - (trade_price[-1] if trade_price else None))
    
    elif row['Position'] < 0:
        
        # Find the unrealized SHORT pnl via (most recent trade price added to trade_price list - Period Close)
        signals_df.at[index, 'trade_delta'] = ((trade_price[-1] if trade_price else None) - row['close'])
    
# Populate unrealized value of trade        
    row['trade_value'] = round(row['trade_delta'] * adjuster,0)
 
    
# Populate REALIZED and UNREALIZED value of account at times of execution
    
    # If Position indicates SHORT (flip from long -> realize long trade PnL)
    if row['Entry/Exit'] < 0:
    
        # Set realized pnl variable. REALIZED $ LONG pnl via (execution Period Close - most recent trade price added to trade_price list)
        realized_pnl = round((row['close'] - (trade_price[-2] if trade_price else None)) * adjuster,0)
        
        # Append variable value to list
        realized.append(realized_pnl)
        
        # Insert to dataframe column at the row of the execution
        signals_df.at[index, 'realized_pnl'] = realized_pnl
        
        # Use new realized balance as basis for ongoing unrealized value
        row['unrealized_acct_val'] = sum(realized) #[-1] if trade_price else None
    
    # If Position indicates LONG (flip from short -> realize short trade PnL)
    elif row['Entry/Exit'] > 0:
        
        # Set realized pnl variable. REALIZED $ SHORT pnl via (most recent trade price added to trade_price list - execution Period Close)
        realized_pnl = round(((trade_price[-2] if trade_price else None) - row['close']) * adjuster,0)
                
        # Append variable value to list
        realized.append(realized_pnl)
        
        # Insert to dataframe column at the row of the execution
        signals_df.at[index, 'realized_pnl'] = realized_pnl
        
        # Use new realized balance as basis for ongoing unrealized value
        row['unrealized_acct_val'] = sum(realized) #[-1] if trade_price else None
    
    # On non-execution rows where there at least one value is in 'realized' list,
    elif len(realized) > 0:
        # Set unrealized_acct_val to realized pnl up to that point PLUS current trade_value 
        row['unrealized_acct_val'] = row['trade_value'] + sum(realized)
    else:
        # Before the first trade is closed (nothing in 'realized' list), just initial_capital and trade_value
        row['unrealized_acct_val'] = initial_capital + row['trade_value']

# Populate Cumulative Realized Account Value        
    row['cum_realized_acct_val'] = initial_capital + signals_df['realized_pnl'].sum()


In [13]:
# Calculate hourly returns
signals_df['Hourly_Returns'] = signals_df['unrealized_acct_val'].pct_change()

#Calculate Cumulative returns
###Using Cumsum instead of Cumprod (the latter I believe would be used if we were buying $ amounts + could size progressively, instead of trading a discrete number of contracts)
signals_df['Cumulative_Returns'] = signals_df['Hourly_Returns'].cumsum()


In [14]:
signals_df.head(5)

,close,VWAP,200EMA,Signal,Entry/Exit,Position,Entry/Exit Position,trade_price,trade_delta,trade_value,unrealized_acct_val,realized_pnl,cum_realized_acct_val,Hourly_Returns,Cumulative_Returns
time,,,,,,,,,,,,,,,
2021-01-14 09:00:00-05:00,35.87,35.756447,34.091800,1.0,NaN,10.0,NaN,35.87,0.00,0.0,50000.0,0.0,50000.0,NaN,NaN
2021-01-14 10:00:00-05:00,35.61,35.754863,34.106906,1.0,0.0,10.0,0.0,0.00,-0.26,-2600.0,47400.0,0.0,50000.0,-0.052000,-0.052000
2021-01-14 11:00:00-05:00,35.92,35.762846,34.124947,1.0,0.0,10.0,0.0,0.00,0.05,500.0,50500.0,0.0,50000.0,0.065401,0.013401
2021-01-14 12:00:00-05:00,36.27,35.794224,34.146291,1.0,0.0,10.0,0.0,0.00,0.40,4000.0,54000.0,0.0,50000.0,0.069307,0.082708
2021-01-14 13:00:00-05:00,36.29,35.835392,34.167621,1.0,0.0,10.0,0.0,0.00,0.42,4200.0,54200.0,0.0,50000.0,0.003704,0.086411


In [15]:
# Visualize the entry positions relative to the Portfolio Total
long_trade = signals_df[signals_df["Entry/Exit"] == 2.0]["unrealized_acct_val"].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False,
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize the exit positions relative to the Portfolio Total
short_trade = signals_df[signals_df["Entry/Exit"] == -2.0]["unrealized_acct_val"].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize Portfolio Total for the investment
total_portfolio_value = signals_df[["unrealized_acct_val"]].hvplot(
    line_color="lightgray", 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Overlay the entry, exit and total_portfolio_value plots
portfolio_entry_exit_plot = total_portfolio_value * long_trade * short_trade
portfolio_entry_exit_plot.opts(
    title="CL Algo PnL",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                        :Curve   [time]   (unrealized_acct_val)
   .Scatter.Unrealized_acct_val.I  :Scatter   [time]   (unrealized_acct_val)
   .Scatter.Unrealized_acct_val.II :Scatter   [time]   (unrealized_acct_val)

In [16]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [17]:
# Calculate the Annualized return metric
portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Hourly_Returns'].mean() * 252 * 22
)

# Calculate the Cumulative returns metric
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Cumulative_Returns'][-1]

# Calculate the Annual volatility metric
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Hourly_Returns'].std() * np.sqrt(252*22)
)

# Calculate the Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Hourly_Returns'].mean() * 252 * 22) / (
    signals_df['Hourly_Returns'].std() * np.sqrt(252 * 22)
)

# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[['Hourly_Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Hourly_Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Hourly_Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Hourly_Returns'].mean() * 252 * 22

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252 * 22)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio


In [18]:
portfolio_evaluation_df

,Backtest
Annualized Return,614.496583
Cumulative Returns,1633.115558
Annual Volatility,1181.726064
Sharpe Ratio,0.519999
Sortino Ratio,1.484167


### Trade Summary (NEEDS WORK)

In [26]:
# Initialize the trade evaluation DataFrame
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Name', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_holding = abs(row['cum_realized_acct_val'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_holding = abs(row['cum_realized_acct_val'])
        exit_share_price = row['close']
        profit_loss =  exit_share_price - entry_share_price
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Name':contract_name,
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Review the DataFrame
trade_evaluation_df

C:\Users\giffo\AppData\Local\Temp\ipykernel_18528\225857587.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\giffo\AppData\Local\Temp\ipykernel_18528\225857587.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\giffo\AppData\Local\Temp\ipykernel_18528\225857587.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
C:\Users\giffo\AppData\Local\Temp\ipykernel_18528\225857587.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.

,Name,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,CL1!,,2021-01-22 05:00:00-05:00,0,0.00,34.76,0.0,1352198.76,34.76
1,CL1!,2021-01-25 03:00:00-05:00,2021-01-25 09:00:00-05:00,20.0,35.61,35.30,30402.0,963760.60,-0.31
2,CL1!,2021-01-25 16:00:00-05:00,2021-01-27 18:00:00-05:00,20.0,35.84,35.39,21903.0,615892.17,-0.45
3,CL1!,2021-01-27 20:00:00-05:00,2021-01-28 18:00:00-05:00,20.0,35.68,35.20,14503.0,341545.60,-0.48
4,CL1!,2021-01-29 09:00:00-05:00,2021-01-31 18:00:00-05:00,20.0,35.91,34.95,2604.0,244475.25,-0.96
5,CL1!,2021-02-01 09:00:00-05:00,2021-03-01 18:00:00-05:00,20.0,35.48,43.22,12294.0,2813535.56,7.74
6,CL1!,2021-03-03 20:00:00-05:00,2021-03-16 09:00:00-04:00,20.0,44.35,47.12,53799.0,3840091.52,2.77
7,CL1!,2021-03-16 18:00:00-04:00,2021-03-17 08:00:00-04:00,20.0,47.96,47.03,73097.0,3000419.94,-0.93
8,CL1!,2021-03-29 18:00:00-04:00,2021-03-30 09:00:00-04:00,20.0,45.06,43.57,83496.0,2988771.29,-1.49
9,CL1!,2021-04-04 18:00:00-04:00,2021-04-05 05:00:00-04:00,20.0,44.16,43.09,62698.0,2240636.91,-1.07
